<p style="font-weight:bold;"> <span style="font-size: 36px"> Application Message </span> </p>

In [1]:
#r "nuget:Systemorph.Activities,1.2.3"

In [1]:
#!import "../Constants/Validations"

In [1]:
using Microsoft.Extensions.Logging;

public class ApplicationMessage {
    private static ILogger log;
    
    private static IActivityVariable activity;
    
    public static void Configure (ILogger log, IActivityVariable activity) 
    {
        ApplicationMessage.log = log;
        ApplicationMessage.activity = activity;
    }
    
    public static Object Log (Error e, params string[] s) { log.LogError(Get(e,s)); return null; }
    
    public static Object Log (Warning w, params string[] s) { log.LogWarning(Get(w,s)); return null; }
    
    public static bool HasErrors () => activity.HasErrors();
}

In [1]:
ApplicationMessage.Configure(Log, Activity)

# Utils

In [1]:
public static bool HasErrors (this ActivityLog[] logs) {
    foreach (var log in logs)
        if(log.Errors.Any())
            return true;
    return false;
}

In [1]:
public static ActivityLog Merge (this ActivityLog a, ActivityLog b) {
    return a with { 
        StartDateTime  = a.StartDateTime < b.StartDateTime ? a.StartDateTime : b.StartDateTime,
        FinishDateTime = a.FinishDateTime > b.FinishDateTime ? a.FinishDateTime : b.FinishDateTime,
        Errors         = a.Errors.Concat(b.Errors).ToList(),
        Warnings       = a.Warnings.Concat(b.Warnings).ToList(),
        Infos          = a.Infos.Concat(b.Infos).ToList(),
        Status         = a.Status == ActivityLogStatus.Failed || b.Status == ActivityLogStatus.Failed ? ActivityLogStatus.Failed : ActivityLogStatus.Succeeded
    };
}

public static ActivityLog Merge (this ActivityLog[] logs) 
{
    if(logs == null || logs.Length == 0) return null;
    return logs.Aggregate((x, y) => x.Merge(y));
}

public static ActivityLog Merge (this ActivityLog[] logs, ActivityLog log) => logs.Merge().Merge(log);
public static ActivityLog Merge (this ActivityLog log, ActivityLog[] logs) => log.Merge(logs.Merge());